In [2]:
'''
初コンペ
二値分類のタスク
決定木を使ってみた
パラメータはとりあえず関係ありそうなものを入れてるだけ
'''
#ライブラリのインポート

#pandasのインポート
import pandas as pd

#グラフ表示のためのインポート
import matplotlib.pyplot as plt

#線形回帰モデルのインポート
from sklearn import linear_model 
import sklearn.model_selection

#決定木モデルのインポート
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

In [3]:
#各自データのインストール
test = pd.read_csv('saiketsu_test .csv')
train = pd.read_csv('saiketsu_train .csv')

In [4]:
#格納されているデータの確認
test.head()

,id,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,1,67,1,56,0,60,0,255844.0310,0.824480,140,0,0,147
1,2,58,0,320,0,38,0,163434.8513,5.281481,134,0,0,207
2,4,67,1,737,0,60,0,255278.8436,0.841388,140,0,0,113
3,5,48,0,231,1,60,0,500149.7955,0.742807,139,1,1,215
4,6,59,0,320,0,35,0,245489.2530,1.082861,132,1,0,215


In [5]:
#格納されているデータの確認
train.head()

,id,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,target
0,0,56,1,5882,0,38,0,185315.0104,0.844417,134,1,0,10,1
1,3,50,0,1767,1,35,0,371620.3666,0.837448,136,0,0,187,1
2,7,41,0,250,1,30,0,372810.2084,0.819996,139,1,0,29,1
3,11,47,0,582,1,20,0,185780.6482,0.870314,137,0,0,43,0
4,12,60,0,80,0,25,0,265096.3999,0.828112,136,1,0,213,0


In [6]:
train.dtypes

id                            int64
age                           int64
anaemia                       int64
creatinine_phosphokinase      int64
diabetes                      int64
ejection_fraction             int64
high_blood_pressure           int64
platelets                   float64
serum_creatinine            float64
serum_sodium                  int64
sex                           int64
smoking                       int64
time                          int64
target                        int64
dtype: object

In [7]:
#テストデータに欠損があるかどうか
test.isnull().sum()

id                          0
age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
time                        0
dtype: int64

In [8]:
#学習データに欠損があるかどうか 
train.isnull().sum()

id                          0
age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
time                        0
target                      0
dtype: int64

In [9]:
#主要統計値の確認
test.describe()

,id,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
count,1000.00000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,999.26600,58.221000,0.38200,576.722000,0.288000,37.195000,0.245000,277448.502756,1.029092,136.851000,0.756000,0.266000,144.869000
std,577.27564,10.157132,0.48612,908.726074,0.453058,11.479318,0.430302,85625.028062,0.688246,3.651045,0.429708,0.442085,78.195271
min,1.00000,40.000000,0.00000,23.000000,0.000000,17.000000,0.000000,117373.947400,0.627076,116.000000,0.000000,0.000000,7.000000
25%,504.50000,50.000000,0.00000,84.000000,0.000000,25.000000,0.000000,255598.445400,0.793282,136.000000,1.000000,0.000000,79.750000
50%,995.50000,58.000000,0.00000,231.000000,0.000000,35.000000,0.000000,267212.471150,0.839786,136.000000,1.000000,0.000000,172.000000
75%,1487.75000,66.000000,1.00000,582.000000,1.000000,45.000000,0.000000,279969.260275,0.948659,140.000000,1.000000,1.000000,213.000000
max,1999.00000,84.000000,1.00000,7861.000000,1.000000,60.000000,1.000000,859856.405100,9.189842,148.000000,1.000000,1.000000,285.000000


In [10]:
#心不全ではない人の主要統計量
train[train['target'] == 0].describe()

,id,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,target
count,802.000000,802.000000,802.000000,802.000000,802.000000,802.000000,802.000000,802.000000,802.000000,802.000000,802.000000,802.000000,802.000000,802.0
mean,991.683292,57.063591,0.423940,546.165835,0.263092,36.837905,0.205736,280510.198078,0.952887,136.995012,0.763092,0.279302,155.431421,0.0
std,576.697645,9.099242,0.494489,873.775010,0.440587,11.087042,0.404490,80003.521205,0.493703,2.845146,0.425451,0.448936,75.866289,0.0
min,11.000000,39.000000,0.000000,23.000000,0.000000,15.000000,0.000000,140648.818900,0.629178,130.000000,0.000000,0.000000,4.000000,0.0
25%,490.500000,50.000000,0.000000,84.000000,0.000000,25.000000,0.000000,258251.879150,0.789294,136.000000,1.000000,0.000000,90.000000,0.0
50%,974.000000,58.000000,0.000000,232.000000,0.000000,35.000000,0.000000,266543.111550,0.832722,136.000000,1.000000,0.000000,187.000000,0.0
75%,1498.750000,61.000000,1.000000,582.000000,1.000000,43.750000,0.000000,279672.063325,0.920312,139.000000,1.000000,1.000000,213.000000,0.0
max,1998.000000,85.000000,1.000000,5882.000000,1.000000,60.000000,1.000000,824131.916700,5.644760,145.000000,1.000000,1.000000,285.000000,0.0


In [11]:
#心不全の人の主要統計量
train[train['target'] == 1].describe()

,id,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,target
count,198.000000,198.000000,198.000000,198.000000,198.00000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.0
mean,1032.343434,59.934343,0.383838,845.121212,0.29798,36.095960,0.363636,269500.320598,1.343647,136.217172,0.722222,0.161616,125.777778,1.0
std,583.587821,12.298353,0.487552,1270.468220,0.45853,10.839666,0.482265,77222.368195,1.184723,5.041786,0.449039,0.369031,79.077685,0.0
min,0.000000,41.000000,0.000000,54.000000,0.00000,14.000000,0.000000,138173.134500,0.644025,116.000000,0.000000,0.000000,10.000000,1.0
25%,532.500000,49.000000,0.000000,129.000000,0.00000,30.000000,0.000000,245924.126850,0.840683,134.000000,0.000000,0.000000,43.000000,1.0
50%,1095.000000,58.000000,0.000000,579.500000,0.00000,35.000000,0.000000,270778.422650,0.951325,137.000000,1.000000,0.000000,115.000000,1.0
75%,1557.500000,68.000000,1.000000,835.000000,1.00000,38.000000,1.000000,280363.971450,1.447136,140.000000,1.000000,0.000000,205.000000,1.0
max,1996.000000,85.000000,1.000000,7861.000000,1.00000,70.000000,1.000000,817402.933100,8.994597,146.000000,1.000000,1.000000,278.000000,1.0


In [12]:
#影響が大きそうな変数：high_blood_pressure,smoking

In [13]:
#int型のデータを使って試してみる
target_col = ["high_blood_pressure","smoking","sex","age","anaemia","creatinine_phosphokinase","diabetes","ejection_fraction","serum_sodium","time","platelets","serum_creatinine"]
predict_data = train[ target_col]
predict_data.head

<bound method NDFrame.head of      high_blood_pressure  smoking  sex  age  anaemia  \
0                      0        0    1   56        1   
1                      0        0    0   50        0   
2                      0        0    1   41        0   
3                      0        0    0   47        0   
4                      0        0    1   60        0   
..                   ...      ...  ...  ...      ...   
995                    0        0    1   60        0   
996                    0        0    1   50        1   
997                    1        0    1   51        1   
998                    0        0    1   76        1   
999                    0        0    1   50        0   

     creatinine_phosphokinase  diabetes  ejection_fraction  serum_sodium  \
0                        5882         0                 38           134   
1                        1767         1                 35           136   
2                         250         1                 30           

In [14]:
#線形回帰モデルで学習
model = linear_model.LinearRegression()
X = predict_data
y = train["target"]

In [15]:
#テストのためにデータを分ける
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)
#モデルを学習したデータに当てはめてみる
model.fit(X_train, y_train)

LinearRegression()

In [16]:
#線形回帰で試す
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.15849572904937126
0.022923303483689095


In [17]:
#決定木で試す
model_Tree = DecisionTreeClassifier(random_state=0)
model_Tree.fit(X_train, y_train)
print(model_Tree.score(X_train, y_train))
print(model_Tree.score(X_test, y_test))

1.0
0.76


In [20]:
target_col = ["high_blood_pressure","smoking","sex","age","anaemia","creatinine_phosphokinase","diabetes","ejection_fraction","serum_sodium","time","platelets","serum_creatinine"]
pred_data = test[target_col]

y_pred = model_Tree.predict(pred_data)
print(len(y_pred))

1000


In [22]:
sub = pd.read_csv('sample_submission.csv', encoding = 'UTF-8', names=['id', 'target'])

sub['target'] = list(map(int, y_pred))
sub.to_csv('submission.csv', index=False, header=False)